<img src="dd_logo.png" />

# Distributed Tracing with Datadog APM

Now that we've got the basics for how traces work, it's time to use a real world server. In our case, we'll use `docker-compose` to load up a running instance, and manipulate things from there.

Before we get started, be sure to check out the repo that goes along with this. It'll have everything you need.

If you're running this notebook locally, you should already be good. Otherwise, you'll want to:

```bash
$ git clone https://github.com/burningion/dash-apm-workshop
$ cd dash-apm-workshop
$ jupyter notebook
```

## Becoming Acquainted with the Example Project

Before we instrument our example project, let's become familiar with its architecture.

Open up a new terminal, and spin up the docker-compose of the repo:

```bash
$ DD_API_KEY=<YOUR_API_KEY> STEP=1 docker-compose up
```


Running docker-compose up spins up all the containers for our infrastructure. 

In this case, we're using docker-compose to spin up two microservices. For now, we've got an API that sits in front of our microservice, and of course, a microservice. 

The first example is already set up with a basic tracer initialized, so by putting in our key, we can already see traces being sent.

Let's try our first `curl` request to the API, and see if we can trace our request across both services:

In [ ]:
!curl http://localhost:5000/think/?subject=war

In [ ]:
!curl http://localhost:5000/think/?subject=music

In [ ]:
!curl http://localhost:5000/think/?subject=mankind

## Viewing Our Traces to See Our Infrastructure

Now that we've got a few requests that have been sent, we can take a look at the Datadog APM dashboard, and see what's going on with our service.

Looking at the dashboard, it appears our trace which should be a single trace is broken out into two.

Our customer facing API is hitting the `thinker` microservice, but the `trace` isn't being propagated across both.

This is an example of what we'll encounter when we first get tracing installed on our systems. We now need to instrument them, adding in our links.


## Continuing Our Traces Across Services Manually

Indeed, if we look at the `thinker.py` file, we can see that even though our `think` function is wrapped in a trace, we're not continuing or checking for any exisisting spans. 

In order to do that within Flask, we'll need to add a check in our request headers for our `X-Datadog-Trace-Id` and `X-Datadog-Parent-Id`, injecting our `trace_id` and `parent_id` if there are either.

Our Python code becomes the following:

```python
@app.route('/')
def think_microservice():
    # continue the span from the called service
    trace_id = flask_request.headers.get("X-Datadog-Trace-Id")
    parent_id = flask_request.headers.get("X-Datadog-Parent-Id")
    if trace_id and parent_id:
        span = tracer.current_span()
        span.trace_id = int(trace_id)
        span.parent_id = int(parent_id)

    subject = flask_request.args.get('subject')
    thoughts = think(subject)
    return Response(thoughts, mimetype='application/json')
```

Notice the `think` function that gets called has a Python decorator. It's wrapping the function call with a span, and inserting the `subject` of the think call into the span's `tag`:


```python
@tracer.wrap(name='think')
def think(subject):
    tracer.current_span().set_tag('subject', subject)

    sleep(0.5)
    return thoughts[subject]
```

If we want, we can restart our containers now, and see how things look with requests being passed across services:

In [ ]:
!docker-compose down
!DD_API_KEY=<YOUR_API_KEY> STEP=2 docker-compose up

## Viewing Cross Service Spans

In order to view our cross service spans, we'll first need to generate some more requests, creating new traces to be sent back to Datadog.

Let's do that now:

In [ ]:
!curl http://localhost:5000/think/?subject=war

Let's try generating an error in our application:

In [ ]:
!curl http://localhost:5000/think/?subject=peace

In [ ]:
!curl http://localhost:5000/think/?subject=mankind

Now, when we switch over to view our traces in Datadog, we see them coming in as a single span, traversing our microservices.

But if you looked closely, you'll see that we have a library that can be instrumented by Datadog, but isn't.

That's the `requests` library, that's used to send our requests across from one microservice to the other. 

## Automatic Instrumentation of Libraries

If we use Datadog's Python library function `patch`, we can also instrument the `requests` library, and get all the metadata of our request along with the information set.

Our original API becomes:

```python
import blinker as _
import requests

from flask import Flask, Response
from flask import jsonify
from flask import request as flask_request

from ddtrace import tracer, patch
from ddtrace.contrib.flask import TraceMiddleware


# Tracer configuration
tracer.configure(hostname='agent')
# also patch the requests library
patch(requests=True)

app = Flask('api')
traced_app = TraceMiddleware(app, tracer, service='thinker-api')


@app.route('/think/')
def think_handler():
    thoughts = requests.get('http://thinker:5001/', headers={
        'x-datadog-trace-id': str(tracer.current_span().trace_id),
        'x-datadog-parent-id': str(tracer.current_span().span_id),
    }, params={
        'subject': flask_request.args.getlist('subject', str),
    }).content
    return Response(thoughts, mimetype='application/json')
```
